In [1]:
import os
import pandas as pd
from settings import *

In [2]:
img_files = []
labels = []
label_code = []

for folder in FOLDERS:
    fname = os.path.join(IMAGE_DIR, folder)
    
    for im in os.listdir(fname):
        impath = os.path.join(fname, im)
        
        img_files.append(impath)
        labels.append(folder)
        label_code.append(FOLDERS.index(folder))

In [3]:
import pandas as pd

dataset = pd.DataFrame(dict(filename=img_files, label=labels, code=label_code))

In [4]:
dataset

,filename,label,code
0,images/happy/6954267558_0dc33282db_b.jpg,happy,0
1,images/happy/814630827_5f07025901_b.jpg,happy,0
2,images/happy/12526134885_dc4d61d636_b.jpg,happy,0
3,images/happy/14692172321_584a9df5a7_b.jpg,happy,0
4,images/happy/3850905959_71e290209d_b.jpg,happy,0
...,...,...,...
13660,images/relaxed/41502236_bef18fbeea_b.jpg,relaxed,2
13661,images/relaxed/10755758324_627817f7c3_b.jpg,relaxed,2
13662,images/relaxed/41738301434_3dcd69ef8c_b.jpg,relaxed,2
13663,images/relaxed/10461793333_50c543543c_b.jpg,relaxed,2


In [5]:
import torch

In [6]:
torch.manual_seed(0)

In [7]:
torch.cuda.is_available()

False

In [8]:
torch.backends.mps.is_available()

True

In [9]:
device = torch.device(DEVICE)

In [10]:
from torchvision.io import read_image
from torchvision import transforms as T
from torch.utils.data import Dataset
import math

class DogDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.augments = [T.RandomHorizontalFlip(1), T.RandomRotation(90), T.AutoAugment()]
        self.normalize = T.Compose([
            T.ConvertImageDtype(torch.float),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

    def __len__(self):
        return self.dataset.shape[0] * (len(self.augments) + 1)
    
    def classes(self):
        return self.dataset["code"].unique()

    def __getitem__(self, idx):
        augment = math.floor(idx / self.dataset.shape[0])
        idx = idx % self.dataset.shape[0]
        
        row = self.dataset.iloc[idx,:]
            
        img_path = row["filename"]
        image = read_image(img_path)
        
        label = row["code"]
        
        if augment > 0:
            image = self.augments[augment - 1].forward(image)
        
        image = self.normalize(image)

        return image, int(label), img_path

In [11]:
data = DogDataset(dataset)

In [12]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size], generator=torch.Generator().manual_seed(1))

In [13]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64
EPOCHS = 50

train = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, classes):
        super(NeuralNetwork, self).__init__()
        
        self.bn = nn.BatchNorm2d(64)
        
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 64, 2),
            self.bn,
            nn.Conv2d(64, 64, 2),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2, 2)
        )
        
        self.dense = nn.Sequential(
            nn.Linear(64 * 46 * 46, 64),
            nn.Linear(64, len(classes))
        )

    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x, 1) 
        x = self.dense(x)
        return x

In [15]:
labels = data.classes()

In [16]:
model = NeuralNetwork(labels).to(device)

In [17]:
model

NeuralNetwork(
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=135424, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

In [19]:
size = len(train.dataset)

for epoch in range(EPOCHS):
    for batch, (images, labels, img_paths) in enumerate(train): 
        optimizer.zero_grad()
        
        images = images.to(device)
        pred = model(images.float())
        
        labels = labels.to(device)
        loss = loss_fn(pred, labels)

        loss.backward()
        optimizer.step()

    loss, current = loss.item(), batch * len(images)
    print(f"loss: {loss:>7f}  [{epoch}]")

loss: 1.188425  [0]
loss: 0.948275  [1]
loss: 1.140245  [2]
loss: 1.018567  [3]
loss: 0.909801  [4]
loss: 1.117078  [5]
loss: 0.844888  [6]
loss: 1.015936  [7]
loss: 0.928844  [8]
loss: 0.855809  [9]
loss: 0.765995  [10]
loss: 1.156586  [11]
loss: 0.951389  [12]
loss: 0.873437  [13]
loss: 0.705562  [14]
loss: 0.962199  [15]
loss: 0.542148  [16]
loss: 0.683009  [17]
loss: 0.918836  [18]
loss: 0.848787  [19]
loss: 1.233186  [20]
loss: 0.778633  [21]
loss: 1.013639  [22]
loss: 0.847537  [23]
loss: 0.751866  [24]
loss: 0.468113  [25]
loss: 0.792603  [26]
loss: 0.653036  [27]
loss: 0.814533  [28]
loss: 0.532518  [29]
loss: 0.701030  [30]
loss: 0.925584  [31]
loss: 0.675262  [32]
loss: 1.048318  [33]
loss: 0.407726  [34]
loss: 0.735102  [35]
loss: 0.689762  [36]
loss: 1.006736  [37]
loss: 0.689685  [38]
loss: 0.637291  [39]
loss: 0.621846  [40]
loss: 0.808835  [41]
loss: 0.740050  [42]
loss: 0.293002  [43]
loss: 0.607648  [44]
loss: 0.504599  [45]
loss: 0.538745  [46]
loss: 0.703347  [47]
lo

In [31]:
torch.save(model, 'dog_model.pth')

In [21]:
#model = torch.load('dog_model.pth')

In [22]:
all_preds = list()
all_labels = list()
all_paths = list()

with torch.no_grad():
    for batch, (images, labels, img_paths) in enumerate(test):
        
        images = images.to(device)
        outputs = model(images.float())
        
        _, preds = torch.max(outputs.data, 1)
        
        all_labels.append(labels)
        all_preds.append(preds)
        all_paths.append(img_paths)

In [23]:
import numpy as np

preds = np.concatenate([p.cpu().numpy() for p in all_preds])
labels = np.concatenate([p.cpu().numpy() for p in all_labels])
paths = np.concatenate([p for p in all_paths])

In [24]:
((preds == labels).sum()) / len(labels)

0.5694291986827662

In [35]:
predictions = pd.DataFrame(dict(pred=preds, label=labels, path=paths))
predictions["correct"] = (predictions["pred"] == predictions["label"])

In [36]:
predictions["prediction"] = predictions["pred"].apply(lambda x: FOLDERS[int(x)])

In [37]:
predictions["actual"] = predictions["label"].apply(lambda x: FOLDERS[int(x)])

In [39]:
predictions.groupby("prediction").apply(lambda x: x["correct"].sum() / x.shape[0])

prediction
happy      0.604082
relaxed    0.501872
sad        0.650061
dtype: float64

In [40]:
disp = predictions.iloc[:50,:].copy()

In [41]:
disp

,pred,label,path,correct,prediction,actual
0,0,0,images/happy/2612556559_64038d87f1_b.jpg,True,happy,happy
1,0,2,images/relaxed/37771192115_716b34be20_b.jpg,False,happy,relaxed
2,2,1,images/sad/3233981099_a15902220f_b.jpg,False,relaxed,sad
3,0,0,images/happy/24352178056_21a4f44910_b.jpg,True,happy,happy
4,2,0,images/happy/16990443072_a2a2bfb629_b.jpg,False,relaxed,happy
5,0,2,images/relaxed/4695162062_7ed8f1ed1c_b.jpg,False,happy,relaxed
6,0,0,images/happy/16576096110_6a91ef9c97_b.jpg,True,happy,happy
7,2,2,images/relaxed/6839270867_0be8455a8b_b.jpg,True,relaxed,relaxed
8,2,0,images/happy/9297928546_a8a17a634d_b.jpg,False,relaxed,happy
9,0,0,images/happy/477297299_1a807a734b_b.jpg,True,happy,happy


In [42]:
def image_formatter(path):
    return f'<img src="{path}">'

In [43]:
disp.style.format({'path': image_formatter})

,pred,label,path,correct,prediction,actual
0,0,0,,True,happy,happy
1,0,2,,False,happy,relaxed
2,2,1,,False,relaxed,sad
3,0,0,,True,happy,happy
4,2,0,,False,relaxed,happy
5,0,2,,False,happy,relaxed
6,0,0,,True,happy,happy
7,2,2,,True,relaxed,relaxed
8,2,0,,False,relaxed,happy
9,0,0,,True,happy,happy
